In [1]:
from captcha_utils.generator import *
from models.cnn_ctc import *


train_path = 'E:\\traindata\\captcha_origin\\train'
validate_path = 'E:\\traindata\\captcha_origin\\test'
img_w = 200
img_h = 60
channel = 1
downsample_factor = 4


model, test_func = icp_model()
# image_gen = TextImageGenerator(train_path, validate_path, img_w, img_h, channel, downsample_factor)
# 使用随机的训练集
image_gen = RandomTextImageGenerator(train_path, validate_path, img_w, img_h, channel, downsample_factor)

# 测试
# test, _ = next(image_gen.get_next_train(10))
# test, _ = next(image_gen.get_next_val(5))
# x_input = test['the_input']
# print(x_input.shape)
# print(image_gen.cur_train_idx)
# print(image_gen.cur_vald_idx)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


E:\Github\machine_learning\image\qrcode\models\cnn_ctc.py:64: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("th..., outputs=Tensor("so...)`
  base_model = Model(input=input_data, output=y_pred)


In [2]:
import keras.callbacks
import datetime

class VizCallback(keras.callbacks.Callback):

    def __init__(self, run_name, test_func, text_img_gen, output_dir, num_display_words=6):
        self.test_func = test_func
        self.output_dir = os.path.join(
            output_dir, run_name)
        self.text_img_gen = text_img_gen
        self.num_display_words = num_display_words
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

    def on_epoch_end(self, epoch, logs={}):
        acc_ratio = 0
        if epoch % 10 == 0:  # 每10个周期计算一次正确率
            word_batch = next(self.text_img_gen)[0]
            res = decode_batch(self.test_func, TextImageGenerator.labels_to_text, word_batch['the_input'])
            acc = 0
            total = word_batch['the_input'].shape[0]
            for i in range(total):
                if word_batch['source_str'][i].lower() == res[i].lower():
                    acc += 1
            acc_ratio = 100 * acc / total
            print('正确率: %0.5f' % acc_ratio)
        if acc_ratio > 50 or epoch % 100 == 0 :
            self.model.save_weights(os.path.join(self.output_dir, 'weights%02d_acc_%0.5f.h5' % (epoch, acc_ratio)))
        word_batch = next(self.text_img_gen)[0]
        res = decode_batch(self.test_func, TextImageGenerator.labels_to_text, word_batch['the_input'][0:self.num_display_words])
        if word_batch['the_input'][0].shape[0] < 256:
            cols = 2
        else:
            cols = 1
        for i in range(self.num_display_words):
            plt.subplot(self.num_display_words // cols, cols, i + 1)
            if K.image_data_format() == 'channels_first':
                the_input = word_batch['the_input'][i, 0, :, :]
            else:
                the_input = word_batch['the_input'][i, :, :, 0]
            plt.imshow(the_input, cmap='Greys_r')
            plt.xlabel('T = \'%s\' Decoed = \'%s\'' % (word_batch['source_str'][i], res[i]))
        plt.savefig(os.path.join(self.output_dir, 'e%02d.png' % (epoch)))
        plt.close()

        
vic_gen = RandomTextImageGenerator(train_path, validate_path, img_w, img_h, channel, downsample_factor)
vic = VizCallback(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S'), test_func, vic_gen.get_next_val(), 
                  output_dir='E:\\Github\\machine_learning\\image\\qrcode\\logs')

In [ ]:
callbacks = [vic]
# 自定义optimizer
optimizer = SGD(lr=0.0005, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)

train(model, image_gen.get_next_train(100), image_gen.get_next_val(400), callbacks=callbacks, epochs=701,
      optimizer=optimizer,
      weight='E:\\Github\\machine_learning\\image\\qrcode\\logs\\2019_06_02_12_34_30\\weights700_acc_61.00000.h5')

Epoch 1/701
100/100 [==============================] - 74s 744ms/step - loss: 0.2473 - val_loss: 2.3273
正确率: 56.00000
Epoch 2/701
100/100 [==============================] - 66s 664ms/step - loss: 0.2513 - val_loss: 2.1825
Epoch 3/701
100/100 [==============================] - 69s 693ms/step - loss: 0.2628 - val_loss: 2.3562
Epoch 4/701
100/100 [==============================] - 70s 704ms/step - loss: 0.2505 - val_loss: 2.2663
Epoch 5/701
100/100 [==============================] - 70s 703ms/step - loss: 0.2555 - val_loss: 2.4106
Epoch 6/701
100/100 [==============================] - 70s 697ms/step - loss: 0.2437 - val_loss: 2.3025
Epoch 7/701
100/100 [==============================] - 69s 693ms/step - loss: 0.2540 - val_loss: 2.3713
Epoch 8/701
100/100 [==============================] - 68s 682ms/step - loss: 0.2689 - val_loss: 2.3027
Epoch 9/701
100/100 [==============================] - 68s 680ms/step - loss: 0.2473 - val_loss: 2.2853
Epoch 10/701
100/100 [============================

100/100 [==============================] - 70s 703ms/step - loss: 0.2601 - val_loss: 2.3639
Epoch 79/701
100/100 [==============================] - 71s 708ms/step - loss: 0.2482 - val_loss: 2.2301
Epoch 80/701
100/100 [==============================] - 69s 688ms/step - loss: 0.2579 - val_loss: 2.3442
Epoch 81/701
100/100 [==============================] - 75s 748ms/step - loss: 0.2419 - val_loss: 2.2968
正确率: 58.00000
Epoch 82/701
100/100 [==============================] - 70s 696ms/step - loss: 0.2577 - val_loss: 2.3015
Epoch 83/701
100/100 [==============================] - 72s 722ms/step - loss: 0.2604 - val_loss: 2.3363
Epoch 84/701
100/100 [==============================] - 70s 705ms/step - loss: 0.2689 - val_loss: 2.3411
Epoch 85/701
100/100 [==============================] - 71s 708ms/step - loss: 0.2526 - val_loss: 2.3386
Epoch 86/701
100/100 [==============================] - 67s 666ms/step - loss: 0.2525 - val_loss: 2.3136
Epoch 87/701
100/100 [==============================] 

100/100 [==============================] - 64s 643ms/step - loss: 0.2459 - val_loss: 2.2612
Epoch 155/701
100/100 [==============================] - 64s 644ms/step - loss: 0.2626 - val_loss: 2.3164
Epoch 156/701
100/100 [==============================] - 65s 646ms/step - loss: 0.2490 - val_loss: 2.3269
Epoch 157/701
100/100 [==============================] - 64s 642ms/step - loss: 0.2436 - val_loss: 2.3809
Epoch 158/701
100/100 [==============================] - 64s 644ms/step - loss: 0.2535 - val_loss: 2.2842
Epoch 159/701
100/100 [==============================] - 65s 648ms/step - loss: 0.2599 - val_loss: 2.2959
Epoch 160/701
100/100 [==============================] - 64s 644ms/step - loss: 0.2509 - val_loss: 2.2773
Epoch 161/701
100/100 [==============================] - 64s 642ms/step - loss: 0.2375 - val_loss: 2.2902
正确率: 55.00000
Epoch 162/701
100/100 [==============================] - 64s 636ms/step - loss: 0.2637 - val_loss: 2.3164
Epoch 163/701
100/100 [=======================

In [ ]:
import keras.preprocessing.image

path = 'E:\\traindata\\captcha_origin\\test\\00e601544fa447d18ae53161c2419c45_6BE45U.png'
img = cv2.imread(path)

img_rotation = keras.preprocessing.image.random_rotation(img, 0)

plt.subplot(2, 1, 1)
plt.imshow(img)
plt.subplot(2, 1, 2)
plt.imshow(img_rotation)